In [3]:
# Importing the required libraries
import requests
import json
import pandas as pd
import time
import datetime
import os
import numpy as np

# Universalis API no key required
# Setting the API Key for XIVAPI from the environment variable XIVAPI_KEY
xivapi_key = os.environ.get('XIVAPI_KEY')

# Setting the URL for the Universalis API
# https://docs.universalis.app/
univeralis_url = 'https://universalis.app/api/'

# Setting the URL for the XIVAPI
# https://xivapi.com/docs
xivapi_url = 'https://xivapi.com/'

In [4]:
tinctures = pd.read_csv('..\\data\\tinctures.csv')
tinctures

,ID,Icon,Name,Url,UrlType,_,_Score
0,39727,/i/020000/020710.png,Grade 8 Tincture of Strength,/Item/39727,Item,item,0
1,39728,/i/020000/020709.png,Grade 8 Tincture of Dexterity,/Item/39728,Item,item,0
2,39729,/i/020000/020707.png,Grade 8 Tincture of Vitality,/Item/39729,Item,item,0
3,39730,/i/020000/020706.png,Grade 8 Tincture of Intelligence,/Item/39730,Item,item,0
4,39731,/i/020000/020708.png,Grade 8 Tincture of Mind,/Item/39731,Item,item,0


In [15]:
# now to get the data for all the marketable consumables
# Initialize an empty df to store all results
tinc_results = pd.DataFrame()

# Initialize the Data center to Aether
# Initialize the histInSeconds to 1 years

world = "Aether"
years = 4
histInSeconds = years*365*24*60*60 # year in seconds

# Loop through all the marketable consumables
# Each request will be for a single consumable
# We have 2321 marketable consumables and we have to limit the requests to 25 per second
# We will use a sleep timer to limit the requests to 25 per second
failed_requests = []

for index, row in tinctures.iterrows():
    # Make the request
    itemID = row['ID']
    response = requests.get(f'https://universalis.app/api/v2/history/{world}/{itemID}?entriesToReturn=999999&entriesWithin={histInSeconds}&minSalePrice=0&maxSalePrice=2147483647')
    # check if the request was successful
    if response.status_code != 200:
        print(f"Request failed for {itemID}")
        # store the itemID in a list to retry later
        failed_requests.append(itemID)
        continue
    response = response.json()
    # normalize the data
    response = pd.json_normalize(response)
    # concatenate the data to the all_results df
    tinc_results = pd.concat([tinc_results, response,], ignore_index=True)
    time.sleep(0.04)


In [16]:
tinc_results

,itemID,lastUploadTime,entries,dcName,regularSaleVelocity,nqSaleVelocity,hqSaleVelocity,stackSizeHistogram.1,stackSizeHistogram.2,stackSizeHistogram.3,...,stackSizeHistogramNQ.43,stackSizeHistogramNQ.52,stackSizeHistogramNQ.53,stackSizeHistogramNQ.56,stackSizeHistogramNQ.73,stackSizeHistogramNQ.22,stackSizeHistogramNQ.44,stackSizeHistogramNQ.47,stackSizeHistogramNQ.62,stackSizeHistogramNQ.83
0,39727,1710952481514,"[{'hq': True, 'pricePerUnit': 2784, 'quantity'...",Aether,32217.000000,78.142860,32138.857000,201,135,1717,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,39728,1710953124128,"[{'hq': True, 'pricePerUnit': 2863, 'quantity'...",Aether,12919.571000,20.142857,12899.429000,101,94,918,...,1.0,1.0,1.0,2.0,1.0,NaN,NaN,NaN,NaN,NaN
2,39729,1710951391617,"[{'hq': True, 'pricePerUnit': 2974, 'quantity'...",Aether,24.714285,0.000000,24.714285,16,7,74,...,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,39730,1710951396745,"[{'hq': True, 'pricePerUnit': 2999, 'quantity'...",Aether,10378.429000,46.142857,10332.286000,62,92,783,...,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN
4,39731,1710949241692,"[{'hq': True, 'pricePerUnit': 2672, 'quantity'...",Aether,13610.714000,21.571428,13589.143000,81,103,934,...,1.0,NaN,2.0,NaN,NaN,2.0,1.0,3.0,1.0,2.0


In [17]:
# using the tinctures df to add the name of the item to the results
tinc_results = tinc_results.merge(tinctures, how='left', left_on='itemID', right_on='ID')
tinc_results = tinc_results.drop(columns=['ID'])
tinc_results

,itemID,lastUploadTime,entries,dcName,regularSaleVelocity,nqSaleVelocity,hqSaleVelocity,stackSizeHistogram.1,stackSizeHistogram.2,stackSizeHistogram.3,...,stackSizeHistogramNQ.44,stackSizeHistogramNQ.47,stackSizeHistogramNQ.62,stackSizeHistogramNQ.83,Icon,Name,Url,UrlType,_,_Score
0,39727,1710952481514,"[{'hq': True, 'pricePerUnit': 2784, 'quantity'...",Aether,32217.000000,78.142860,32138.857000,201,135,1717,...,NaN,NaN,NaN,NaN,/i/020000/020710.png,Grade 8 Tincture of Strength,/Item/39727,Item,item,0
1,39728,1710953124128,"[{'hq': True, 'pricePerUnit': 2863, 'quantity'...",Aether,12919.571000,20.142857,12899.429000,101,94,918,...,NaN,NaN,NaN,NaN,/i/020000/020709.png,Grade 8 Tincture of Dexterity,/Item/39728,Item,item,0
2,39729,1710951391617,"[{'hq': True, 'pricePerUnit': 2974, 'quantity'...",Aether,24.714285,0.000000,24.714285,16,7,74,...,NaN,NaN,NaN,NaN,/i/020000/020707.png,Grade 8 Tincture of Vitality,/Item/39729,Item,item,0
3,39730,1710951396745,"[{'hq': True, 'pricePerUnit': 2999, 'quantity'...",Aether,10378.429000,46.142857,10332.286000,62,92,783,...,NaN,NaN,NaN,NaN,/i/020000/020706.png,Grade 8 Tincture of Intelligence,/Item/39730,Item,item,0
4,39731,1710949241692,"[{'hq': True, 'pricePerUnit': 2672, 'quantity'...",Aether,13610.714000,21.571428,13589.143000,81,103,934,...,1.0,3.0,1.0,2.0,/i/020000/020708.png,Grade 8 Tincture of Mind,/Item/39731,Item,item,0


In [27]:
strength_history = pd.json_normalize(tinc_results['entries'][tinc_results['Name'].str.contains("Strength")].values[0])
#convert the timestamp to a date
strength_history['timestamp'] = pd.to_datetime(strength_history['timestamp'], unit='s')
# save the data to a csv
strength_history.to_csv('..\\data\\strength_tincture_market.csv', index=False)
strength_history


,hq,pricePerUnit,quantity,buyerName,onMannequin,timestamp,worldName,worldID
0,True,2784,99,X'nohsoeh Bryn,False,2024-03-20 15:51:56,Siren,57
1,True,2715,25,Ryu Okami,False,2024-03-20 13:56:40,Gilgamesh,63
2,True,2799,99,Beatrix D'oraguille,False,2024-03-20 13:14:09,Cactuar,79
3,True,2700,33,Faust Faust,False,2024-03-20 13:10:13,Gilgamesh,63
4,True,2715,99,Riena Sun,False,2024-03-20 10:24:49,Gilgamesh,63
...,...,...,...,...,...,...,...,...
173272,True,25000,10,Gwyn Graves,False,2023-05-23 13:53:16,Gilgamesh,63
173273,True,25000,50,Gwyn Graves,False,2023-05-23 13:53:15,Gilgamesh,63
173274,False,6969,3,Xeno Izanagi,False,2023-05-23 12:39:47,Gilgamesh,63
173275,True,19997,15,Sonya Windwalker,False,2023-05-23 11:50:44,Gilgamesh,63


In [28]:
vitality_history = pd.json_normalize(tinc_results['entries'][tinc_results['Name'].str.contains("Vitality")].values[0])
#convert the timestamp to a date
strength_history['timestamp'] = pd.to_datetime(strength_history['timestamp'], unit='s')
# save the data to a csv
vitality_history.to_csv('..\\data\\Vitality_tincture_market.csv', index=False)
vitality_history

,hq,pricePerUnit,quantity,buyerName,onMannequin,timestamp,worldName,worldID
0,True,2974,20,Reina Uberall,False,1710716774,Gilgamesh,63
1,True,1850,30,Elsha Linnette,False,1710421123,Midgardsormr,65
2,True,2996,24,Samara Hana,False,1710418635,Adamantoise,73
3,True,2997,99,Samara Hana,False,1710418634,Adamantoise,73
4,True,2000,33,Sair Hyskaris,False,1710347949,Gilgamesh,63
...,...,...,...,...,...,...,...,...
790,True,25000,3,Jatoro Brathgar,False,1684964107,Jenova,40
791,True,6000,3,Kyro Tachikake,False,1684897691,Jenova,40
792,True,5105,3,Ilyesen Valenroix,False,1684888429,Cactuar,79
793,True,30000,3,Ronin Lothbrook,False,1684874110,Siren,57


In [29]:
dexterity_history = pd.json_normalize(tinc_results['entries'][tinc_results['Name'].str.contains("Dexterity")].values[0])
#convert the timestamp to a date
strength_history['timestamp'] = pd.to_datetime(strength_history['timestamp'], unit='s')
# save the data to a csv
dexterity_history.to_csv('..\\data\\Dexterity_tincture_market.csv', index=False)
dexterity_history

,hq,pricePerUnit,quantity,buyerName,onMannequin,timestamp,worldName,worldID
0,True,2863,51,Lune Cr'est,False,1710952774,Adamantoise,73
1,True,2961,99,Shadow Erebus,False,1710950015,Siren,57
2,True,2759,99,Nunu Nuke,False,1710922473,Jenova,40
3,True,2763,99,Aere Lewde,False,1710915450,Jenova,40
4,True,2582,99,Lil Husl,False,1710914265,Gilgamesh,63
...,...,...,...,...,...,...,...,...
76125,True,14998,33,Shiro Greybourne,False,1684853829,Jenova,40
76126,True,14998,45,Shiro Greybourne,False,1684853828,Jenova,40
76127,True,15000,30,Cipher Mss,False,1684847214,Jenova,40
76128,True,15000,30,Cipher Mss,False,1684847212,Jenova,40


In [30]:
intelligence_history = pd.json_normalize(tinc_results['entries'][tinc_results['Name'].str.contains("Intelligence")].values[0])
#convert the timestamp to a date
strength_history['timestamp'] = pd.to_datetime(strength_history['timestamp'], unit='s')
# save the data to a csv
intelligence_history.to_csv('..\\data\\Intelligence_tincture_market.csv', index=False)
intelligence_history

,hq,pricePerUnit,quantity,buyerName,onMannequin,timestamp,worldName,worldID
0,True,2999,19,Nozomi Duskheart,False,1710941100,Siren,57
1,True,2999,19,Nozomi Duskheart,False,1710941096,Siren,57
2,False,2500,6,Volk Lunam,False,1710936786,Sargatanas,99
3,True,2974,99,Moka Tamamitsune,False,1710933662,Cactuar,79
4,True,2949,30,Moka Tamamitsune,False,1710933651,Cactuar,79
...,...,...,...,...,...,...,...,...
65959,True,15000,10,Vox Veritatis,False,1684861981,Jenova,40
65960,True,30000,33,Camellia Yagami,False,1684854654,Siren,57
65961,True,30000,33,Camellia Yagami,False,1684854653,Siren,57
65962,True,30000,33,Camellia Yagami,False,1684854652,Siren,57


In [31]:
mind_history = pd.json_normalize(tinc_results['entries'][tinc_results['Name'].str.contains("Mind")].values[0])
#convert the timestamp to a date
strength_history['timestamp'] = pd.to_datetime(strength_history['timestamp'], unit='s')
# save the data to a csv
mind_history.to_csv('..\\data\\Mind_tincture_market.csv', index=False)
mind_history

,hq,pricePerUnit,quantity,buyerName,onMannequin,timestamp,worldName,worldID
0,True,2672,99,Camellia Yagami,False,1710918098,Gilgamesh,63
1,True,2672,99,Camellia Yagami,False,1710918097,Gilgamesh,63
2,True,2672,99,Camellia Yagami,False,1710918096,Gilgamesh,63
3,True,2672,99,Camellia Yagami,False,1710918095,Gilgamesh,63
4,True,2600,99,Ryuujin Jizutsu,False,1710916022,Gilgamesh,63
...,...,...,...,...,...,...,...,...
73999,True,13000,27,Halo Nya,False,1684850149,Jenova,40
74000,True,40000,3,Atropia Shade,False,1684848530,Gilgamesh,63
74001,True,24999,9,Atropia Shade,False,1684848529,Gilgamesh,63
74002,True,19000,50,Atropia Shade,False,1684848527,Gilgamesh,63


In [32]:
tinctures

,ID,Icon,Name,Url,UrlType,_,_Score
0,39727,/i/020000/020710.png,Grade 8 Tincture of Strength,/Item/39727,Item,item,0
1,39728,/i/020000/020709.png,Grade 8 Tincture of Dexterity,/Item/39728,Item,item,0
2,39729,/i/020000/020707.png,Grade 8 Tincture of Vitality,/Item/39729,Item,item,0
3,39730,/i/020000/020706.png,Grade 8 Tincture of Intelligence,/Item/39730,Item,item,0
4,39731,/i/020000/020708.png,Grade 8 Tincture of Mind,/Item/39731,Item,item,0
